# Przemysław Kaleta
## Praca domowa nr 2 (09.12.2018)


In [18]:
import pandas as pd
import numpy as np
import os
import sqlite3

Mamy do dyspozycji 7 zbiorów danych z serwisu https://travel.stackexchange.com/. Warto zaznajomić się z tym serwisem żeby zrozumieć jak wyglądają i jak są powiązane dane którymi dysponujemy. Opis poszczególnych tabel można znaleźć na stronie http://www.gagolewski.com/resources/data/travel_stackexchange_com/readme.txt.

Będziemy chcieli znaleźć odpowiedzi na kilka interesujących pytań. Zrobimy to na dwa sposoby: raz używając zapytań SQL, drugi za pomocą biblioteki pandas.

Załadujmy najpierw dane i przyjrzyjmy się jak wyglądają. 

In [19]:
conn = sqlite3.connect('pd2.db')

Comments = pd.read_csv(os.path.join("data", "Comments.csv.gz"))
Votes = pd.read_csv(os.path.join("data", "Votes.csv.gz"))
Badges = pd.read_csv(os.path.join("data", "Badges.csv.gz"))
Tags = pd.read_csv(os.path.join("data", "Tags.csv.gz"))
Posts = pd.read_csv(os.path.join("data", "Posts.csv.gz"))
PostLinks = pd.read_csv(os.path.join("data", "PostLinks.csv.gz"))
Users = pd.read_csv(os.path.join("data", "Users.csv.gz"))


Comments.to_sql("Comments", conn, if_exists='replace')
Votes.to_sql("Votes", conn, if_exists='replace')
Badges.to_sql("Badges", conn, if_exists='replace')
Tags.to_sql("Tags", conn, if_exists='replace')
Posts.to_sql("Posts", conn, if_exists='replace')
PostLinks.to_sql("PostLinks", conn, if_exists='replace')
Users.to_sql("Users", conn, if_exists='replace')

In [20]:
Comments.head()

,CreationDate,Id,PostId,Score,UserId
0,2011-06-21T20:25:14.257,1,1,0,12.0
1,2011-06-21T20:27:35.300,2,1,0,9.0
2,2011-06-21T20:32:23.687,3,1,3,20.0
3,2011-06-21T20:42:08.330,9,25,0,30.0
4,2011-06-21T20:44:09.990,12,26,3,26.0


In [21]:
Posts.head()

,AcceptedAnswerId,AnswerCount,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,LastEditDate,LastEditorUserId,OwnerUserId,ParentId,PostTypeId,Score,Title,ViewCount
0,393.0,4.0,2013-02-25T23:52:47.953,4,NaN,2011-06-21T20:19:34.730,NaN,1,2012-05-24T14:52:14.760,2011-12-28T21:36:43.910,101.0,9.0,NaN,1,8,What are some Caribbean cruises for October?,419.0
1,NaN,7.0,NaN,4,NaN,2011-06-21T20:22:33.760,5.0,2,2014-05-05T22:30:31.273,2012-09-03T16:56:27.963,13.0,13.0,NaN,1,32,How can I find a guide that will take me safel...,1399.0
2,NaN,NaN,NaN,2,NaN,2011-06-21T20:24:28.080,NaN,3,2011-06-21T20:24:28.080,NaN,NaN,9.0,2.0,2,12,NaN,NaN
3,NaN,1.0,NaN,1,NaN,2011-06-21T20:24:57.160,NaN,4,2013-01-09T09:55:22.743,2013-01-09T09:55:22.743,693.0,24.0,NaN,1,8,Does Singapore Airlines offer any reward seats...,243.0
4,770.0,5.0,NaN,0,NaN,2011-06-21T20:25:56.787,2.0,5,2012-10-12T20:49:08.110,2011-12-28T21:36:18.230,101.0,13.0,NaN,1,14,What is the easiest transportation to use thro...,395.0


In [22]:
Users.head()

,AccountId,Age,CreationDate,DisplayName,DownVotes,Id,LastAccessDate,Location,Reputation,UpVotes,Views
0,-1.0,NaN,2011-06-21T15:16:44.253,Community,12646,-1,2011-06-21T15:16:44.253,on the server farm,1,2472,0
1,2.0,40.0,2011-06-21T20:10:03.720,Geoff Dalgas,0,2,2016-05-29T01:18:20.767,"Corvallis, OR",101,1,31
2,7598.0,32.0,2011-06-21T20:11:02.490,Nick Craver,0,3,2015-09-14T12:03:40.247,"Winston-Salem, NC",101,1,14
3,1998.0,31.0,2011-06-21T20:12:21.223,Emmett,0,4,2013-03-01T22:30:26.790,"San Francisco, CA",101,1,10
4,29738.0,NaN,2011-06-21T20:15:31.410,Kevin Montrose,0,5,2017-02-07T02:24:34.733,"New York, NY, United States",101,1,11


### Zadanie 1
Szukamy użytkowników, którzy zadają najciekawsze, najbardziej lubiane pytania. Wybierzemy 10 użytkowników, którzy w sumie najwięcej razy mieli swoje pytania dodawane do ulubionych (posortowanych po sumarycznej liczbie dodań do ulubionych). Do tego zobaczymy wiek i miejsce zamieszkania tych użytkowników i liczbę 'lajków' swojego najbardziej lubianego pytania i jego tytuł. 

Zwróćmy uwagę, że osoba, która dostała w sumie najwięcej lajków nie zadała najbardziej lubianego pytania. Prawie dwa razy więcej lajków miało pytanie drugiego z kolei użytkownika zatytułowane: *'OK we're all adults here, so really, how on earth should I use a squat toilet?'.* Z zupełnie niezrozumiałych dla mnie powodów trzecia w rankingu jest osoba, która zadała pytanie: *'How to avoid drinking vodka?'*.

In [23]:
command1 = """
SELECT
    Users.DisplayName,
    Users.Age,
    Users.Location,
    SUM(Posts.FavoriteCount) AS FavoriteTotal,
    Posts.Title AS MostFavoriteQuestion,
    MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10
"""
pd.read_sql_query(command1, conn)

,DisplayName,Age,Location,FavoriteTotal,MostFavoriteQuestion,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",467.0,Tactics to avoid getting harassed by corrupt p...,42.0
1,hippietrail,NaN,"Oaxaca, Mexico",444.0,"OK we're all adults here, so really, how on ea...",79.0
2,RoflcoptrException,NaN,None,294.0,How to avoid drinking vodka?,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",221.0,What is the highest viewing spot in London tha...,17.0
4,nsn,NaN,None,214.0,How do airlines determine ticket prices?,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",151.0,Are there other places with gardens like those...,10.0
6,Andrew Grimm,38.0,"Sydney, Australia",120.0,"OK we're all nerds here, so really, how on ear...",8.0
7,VMAtm,33.0,"Tampa, FL, United States",109.0,Is there a good website to plan a trip via tra...,34.0
8,jrdioko,NaN,None,100.0,What is the most comfortable way to sleep on a...,21.0
9,Gayot Fow,NaN,"London, United Kingdom",98.0,Should I submit bank statements when applying ...,18.0


In [24]:
PostsUsers = pd.merge(Posts, Users, left_on="OwnerUserId", right_on="Id")

In [25]:
BestUsers = PostsUsers.loc[PostsUsers.PostTypeId == 1, ["OwnerUserId", "DisplayName", "Age", "Location", "FavoriteCount", "Title"]]\
.sort_values("FavoriteCount", ascending=False)\
.groupby(["OwnerUserId"], as_index=False)\
.agg({"FavoriteCount":[sum, max], "Title":'first'})

BestUsers.columns = ["OwnerUserId", "FavoriteTotal", "MostFavoriteQuestionLikes", "Title"]

BestUsers = BestUsers.sort_values("FavoriteTotal", ascending=False).head(10).reset_index(drop=True)

result = pd.merge(BestUsers, Users.loc[:, ["Id", "DisplayName", "Age", "Location"]], 
                  left_on="OwnerUserId", right_on="Id")\
.loc[:, ["DisplayName", "Age", "Location", "Title", "FavoriteTotal", "MostFavoriteQuestionLikes"]]

result

,DisplayName,Age,Location,Title,FavoriteTotal,MostFavoriteQuestionLikes
0,Mark Mayo,37.0,"Sydney, New South Wales, Australia",Tactics to avoid getting harassed by corrupt p...,467.0,42.0
1,hippietrail,NaN,"Oaxaca, Mexico","OK we're all adults here, so really, how on ea...",444.0,79.0
2,RoflcoptrException,NaN,NaN,How to avoid drinking vodka?,294.0,29.0
3,JonathanReez,26.0,"Prague, Czech Republic",What is the highest viewing spot in London tha...,221.0,17.0
4,nsn,NaN,NaN,How do airlines determine ticket prices?,214.0,40.0
5,Gagravarr,NaN,"Oxford, United Kingdom",Are there other places with gardens like those...,151.0,10.0
6,Andrew Grimm,38.0,"Sydney, Australia","OK we're all nerds here, so really, how on ear...",120.0,8.0
7,VMAtm,33.0,"Tampa, FL, United States",Is there a good website to plan a trip via tra...,109.0,34.0
8,jrdioko,NaN,NaN,What is the most comfortable way to sleep on a...,100.0,21.0
9,Gayot Fow,NaN,"London, United Kingdom",Should I submit bank statements when applying ...,98.0,18.0


In [26]:
result.Title[1]

"OK we're all adults here, so really, how on earth should I use a squat toilet?"

W zadaniu tym napotkałem się na kilka **problemów:**

* Na początku gdy chciałem pogrupować po `["DisplayName", "Age", "Location", "OwnerUserId"]` to okazało się że groupby w pandasie na to nie pozwala (a raczej pozwala, ale nie tak jak bym chciał), bo w `Location` znajdują się NA (https://github.com/pandas-dev/pandas/issues/3729). Issue zostało założone w 2013 roku, ale po wielu latach problem nadal nie jest rozwiązany (pull request z maja 2018, zamknięty bez skutku kilkanaście dni temu: https://github.com/pandas-dev/pandas/pull/21669). Żeby rozwiązać problem z tym związany musiałem grupować tylko po ID, a później jeszcze raz mergować z tabelą Users :/


* Przydatną rzeczą jest argument `as_index=False` w groupby (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html), jeżeli chcemy żeby to po czym grupujemy było jedną z kolumn, a nie indeksem (do czego jestem przyzwyczajony w standardowym SQLu). Szkoda tylko że ta opcja nie zachowuje nazwy kolumny i trzeba ją dodawać ręcznie.


* Do wykonywania funkcji agregujących na obiekcie typu GroupBy służy metoda `.agg()`. Jest ona bardzo użyteczna, gdyż możemy agregować wiele kolumn używając wielu funkcji. Część z nich jest już zaimplementowana dla nas i możemy je podać jako napisy (np. 'first', 'mean' itp.), ale niestety dokumentacja (http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.agg.html) nie jest zbyt obszerna i trudno znaleźć coś bardziej szczegółowego na ten temat, w szczególności jakich funkcji możemy użyć.


* To zapytanie SQL jest dość niestandardowe i na początku trudno ogarnąć o co w nim chodzi. Grupujemy tylko po jednej wartości, a wydobywamy nagle wiele kolumn, w tym tytuł i nie bardzo wiadomo na jakiej zasadzie. Okazuje się jednak, że chodzi o wydobycie wartości w wierszach odpowiadająych maksymalnej wartości `FavoriteCount`, po której agregujemy. To właśnie stanowi największą trudność w tym zadaniu: wydobycie tytułu posta odpowiadającego największej liczbie dodań do ulubionych.


* Żeby rozwiązać problem wydobycia tytułu odpowiadającego największej liczbie `FavoriteCount` skorzystałem z faktu, że `groupby()` ma parametr `sort` domyślnie ustawiony na True. Oznacza to, że kolejność wierszy będzie wewnątrz grup taka sama jak w wyjściowej ramce danych. Dzięki temu, gdy najpierw posortowałem ramkę danych po `FavoriteCount` i agregowałem kolumnę `Title` wystarczyło wybrać pierwszą wartość funkcją `first` 😊. Gdyby okazało się, że jest kilka odpowiedzi o tej samej maksymalnej liczbie lajków to nic nie szkodzi: chcemy wybrać tylko jeden tytuł reprezentujący danego użytkownika.

### Zadanie 2
Chcemy znaleźć **pytania, które mają najwięcej odpowiedzi o dodatnim wyniku** (`Score`) (to jest o dodatniej różnicy głosów za minus głosy przeciw). Prawdopodobnie będą to dość proste pytania, na które dużo osób ma coś sensownego do powiedzenia. Posortujemy te pytania po liczbie takich odpowiedzi i wybierzemy 10 z największą liczbą odpowiedzi. Do tego podamy ID tego posta.

In [27]:
command2 = """
SELECT
    Posts.ID,
    Posts.Title,
    Posts2.PositiveAnswerCount
FROM Posts
JOIN (
        SELECT
            Posts.ParentID,
            COUNT(*) AS PositiveAnswerCount
        FROM Posts
        WHERE Posts.PostTypeID=2 AND Posts.Score>0
        GROUP BY Posts.ParentID
    ) AS Posts2
    ON Posts.ID=Posts2.ParentID
ORDER BY Posts2.PositiveAnswerCount DESC
LIMIT 10
"""
pd.read_sql_query(command2, conn)

,Id,Title,PositiveAnswerCount
0,250,Which European cities have bike rental station...,24
1,10,When traveling to a country with a different c...,20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch se...,20
4,3220,Why would you wrap your luggage in plastic?,19
5,43660,Traveling in Europe Solo - 18 years old. Feasi...,19
6,30656,Long-life SIM cards in Europe,18
7,7663,Am I expected to tip wait staff in Europe?,17
8,59128,"Is there a way to prevent ""looking like a tour...",17
9,60446,Is it rude to ask if the food contains pork or...,17


In [28]:
Posts2 = Posts.loc[(Posts.PostTypeId == 2) & (Posts.Score > 0)]\
.groupby(["ParentId"], as_index=False).size().reset_index().astype(int)
Posts2.columns = ["ParentId", "PositiveAnswerCount"]

In [29]:
result = pd.merge(Posts.drop(["ParentId"], axis=1), Posts2, left_on="Id", right_on="ParentId").loc[:, ["ParentId", "Title", "PositiveAnswerCount"]]
result = result.sort_values("PositiveAnswerCount", ascending=False).head(10).reset_index(drop=True)
result

,ParentId,Title,PositiveAnswerCount
0,250,Which European cities have bike rental station...,24
1,10,When traveling to a country with a different c...,20
2,13562,How do you choose a restaurant when travelling?,20
3,48775,How can I deal with people asking to switch se...,20
4,43660,Traveling in Europe Solo - 18 years old. Feasi...,19
5,3220,Why would you wrap your luggage in plastic?,19
6,30656,Long-life SIM cards in Europe,18
7,80329,How to cope with too slow Wi-Fi at hotel?,17
8,7663,Am I expected to tip wait staff in Europe?,17
9,59128,"Is there a way to prevent ""looking like a tour...",17


Rzeczą na którą zwróciłem uwagę w tym zadaniu jest `drop=True` w metodzie `reset_index`. Bez tego dostajemy dodatkową i niepotrzebną kolumnę 'index'.

### Zadanie 3
Pytania z największą liczbą lajków w poszczególnych latach. Dostajemy tytuł, rok oraz liczbę lajków zapytania (w roku 2011 widzimy znajome pytanie z pierwszego zadania).

In [30]:
command3 = """
SELECT
    Posts.Title,
    UpVotesPerYear.Year,
    MAX(UpVotesPerYear.Count) AS Count
FROM (
       SELECT
            PostId,
            COUNT(*) AS Count,
            STRFTIME('%Y', Votes.CreationDate) AS Year
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, Year
        ) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year
"""
pd.read_sql_query(command3, conn)

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,180
4,Immigration officer that stopped me at the air...,2015,119
5,I don't know my nationality. How can I visit D...,2016,135
6,Why prohibit engine braking?,2017,178


In [31]:
UpVotesPerYear = Votes.loc[Votes.VoteTypeId == 2, ["PostId", "CreationDate"]]
UpVotesPerYear["Year"] = UpVotesPerYear.CreationDate.str[0:4]
UpVotesPerYear = UpVotesPerYear.drop(["CreationDate"], axis=1)
UpVotesPerYear = UpVotesPerYear.groupby(["PostId", "Year"], as_index=False).size().reset_index()
UpVotesPerYear.columns = ["PostId", "Year", "Count"]

In [32]:
pd.merge(UpVotesPerYear, Posts.loc[Posts.PostTypeId == 1], left_on="PostId", right_on="Id")\
.loc[:, ["Title", "Year", "Count"]]\
.groupby(["Year"])\
.apply(lambda df: df[df.Count == max(df.Count)])\
.reset_index(drop=True)

,Title,Year,Count
0,"OK we're all adults here, so really, how on ea...",2011,70
1,How to successfully haggle / bargain in markets,2012,37
2,Why are airline passengers asked to lift up wi...,2013,103
3,How do you know if Americans genuinely/literal...,2014,180
4,Immigration officer that stopped me at the air...,2015,119
5,I don't know my nationality. How can I visit D...,2016,135
6,Why prohibit engine braking?,2017,178


**Uwagi**:
* `df.drop()` nie działa in-place
* Nadawanie nazw kolumnom po wykonaniu groupby nie jest najprostsze, trzeba użyć metody `rename` albo ręcznie przypisać nowy atrybut `columns` do ramki ranych
* `apply` jest bardzo fajną funkcją. Pytanie: Co by się stało gdyby było kilka lat z maksymalną liczbą w Councie? Dostalibyśmy większą ramkę danych na wyjściu z kilkoma wierszami odpowiadającymi jednemu roku. Jest to zachowanie, które nam (mi) odpowiada. Wydaje mi się, że w Rze rozwiązałem to w głupszy sposób, który korzystał z `which.max`, czyli wybierał tylko jeden wiersz. W tym konkretnym przypadku output jest taki sam, ale nie musiało tak być.

#### Przykład:

In [33]:
df = pd.DataFrame({"Year":[1996, 1996, 1996, 1970, 1970], "Title":["a", "b", "c", "d", "f"], "Count":[42, 5, 42, 13, 3]})
print(df)
df.groupby(["Year"])\
.apply(lambda df: df[df.Count == max(df.Count)])\
.reset_index(drop=True)

   Count Title  Year
0     42     a  1996
1      5     b  1996
2     42     c  1996
3     13     d  1970
4      3     f  1970


,Count,Title,Year
0,13,d,1970
1,42,a,1996
2,42,c,1996


### Zadanie 4
Wyznaczymy pytania, dla których liczba głosów użytkowników na wybraną przez pytającego odpowiedź najbardziej różni się od liczby głosów na najlepszą odpowiedź.

Można to interpretować jako pytania, w których pytający podjął najbardziej *kontrowersyjną* decyzję w wyborze najlepszej odpowiedzi. Na wyjściu dostajemy ID pytania, tytuł pytania, wynik wybranej odpowiedzi, wynik najlepszej odpowiedzi oraz różnicę między dwoma ostatnimi, przy czym pokażemy tylko pytania z różnicą większą niż 50, posortowane malejąco względem tej różnicy.

In [34]:
command4 = """
SELECT
    Questions.Id,
    Questions.Title,
    BestAnswers.MaxScore,
    Posts.Score AS AcceptedScore,
    BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
        SELECT Id, ParentId, MAX(Score) AS MaxScore
        FROM Posts
        WHERE PostTypeId==2
        GROUP BY ParentId
    ) AS BestAnswers
JOIN (
        SELECT * FROM Posts
        WHERE PostTypeId==1
    ) AS Questions
    ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC
"""
pd.read_sql_query(command4, conn)

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56


In [35]:
BestAnswers = Posts.loc[Posts.PostTypeId == 2, ["ParentId", "Score"]]\
.groupby(["ParentId"])\
.apply(lambda df: df[df.Score == max(df.Score)])\
.rename(columns={"Score":"MaxScore"})\
.reset_index(drop=True)

Questions = Posts.loc[Posts.PostTypeId == 1]

In [36]:
BestAnswers_Questions = pd.merge(BestAnswers, Questions, left_on="ParentId", right_on="Id")
result = pd.merge(BestAnswers_Questions, Posts, left_on="AcceptedAnswerId", right_on="Id")
result["Difference"] = result.MaxScore - result.Score_y
result = result.loc[result.Difference > 50, ["Id_x", "Title_x", "MaxScore", "Score_y", "Difference"]].reset_index(drop=True)
result.columns = ["Id", "Title", "MaxScore", "AcceptedScore", "Difference"]
result = result.sort_values("Difference", ascending=False).reset_index(drop=True)
result

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56


**Uwagi:**
* Zastanawiałem się po co nam kolumna `Id` w BestAnswers i doszedłem do wniosku że jest ona podana tylko dla zmyłki, nie wykorzystujemy jej później w joinowaniu. Co więcej, nie podajemy jej nawet w outpucie jak myślałem na początku, bo przecież podajemy ID w tabeli Questions, a nie ID najlepszej odpowiedzi. Równie dobrze, możemy w BestAnswers nie wybierać Id i tak też postanowiłem ostatecznie zrobić.
* Gdy joinujemy dwie tabele, w których mamy te same kolumny, to do tej lewej zostaje dopisany przyrostek '_x', a do prawej '_y'

### Zadanie 5
Na pytania na stacku można sobie odpowiadać samemu. Można robić tak, że po napotkaniu się na jakiś problem i długim szukaniu odpowiedzi (na przykład w dokumentacji albo w kodzie programu) ktoś postanowi napisać takie pytanie i udzielić odpowiedzi. Może to pomóc ludziom, którzy w przyszłości będą chcieli zadać takie same pytanie.

Właśnie to zjawisko chcemy zbadać w tym zapytaniu. Znajdziemy pytania, użytkowników, którzy pod swoim pytaniem dostali w sumie najwięcej lajków. Zwrócimy tytuł zapytania i całkowity wynik komentarzy użytkownika pod swoim zapytaniem, posortowany malejąco. Wybierzemy 10 największych wyników.

In [37]:
command5 = """
SELECT
    Posts.Title,
    CmtTotScr.CommentsTotalScore
FROM (
        SELECT
            PostID,
            UserID,
            SUM(Score) AS CommentsTotalScore
        FROM Comments
        GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10
"""
pd.read_sql_query(command5, conn)

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,What's the longest scheduled public bus ride i...,25
4,Can I have a watermelon in hand luggage?,25
5,How does President Trump's travel ban affect n...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,Legalities and safety concerns of visiting pro...,20
9,India just demonetized all Rs 500 & 1000 notes...,20


In [38]:
CmtTotScr = Comments.loc[:, ["PostId", "UserId", "Score"]]\
.groupby(["PostId", "UserId"], as_index=False)\
.sum()\
.rename(columns={"Score":"CommentsTotalScore"})

In [39]:
result = pd.merge(CmtTotScr, Posts, left_on=["PostId", "UserId"], right_on=["Id", "OwnerUserId"])
result = result.loc[result.PostTypeId == 1, ["Title", "CommentsTotalScore"]]
result = result.sort_values("CommentsTotalScore", ascending=False).head(10).reset_index(drop=True)

result

,Title,CommentsTotalScore
0,How to intentionally get denied entry to the U...,75
1,How can I deal with people asking to switch se...,32
2,What is France's traditional costume?,26
3,Can I have a watermelon in hand luggage?,25
4,How does President Trump's travel ban affect n...,25
5,What's the longest scheduled public bus ride i...,25
6,Caught speeding 111 Mph (179 km/h) in Californ...,24
7,Returning US Citizen lost passport in Canada,23
8,India just demonetized all Rs 500 & 1000 notes...,20
9,Legalities and safety concerns of visiting pro...,20


### Zadanie 6
Wybierzemy użytkowników, którzy dostali odznaki (`Badges`) pierwszej klasy (złote), ale takie które wśród użytkowników występują nie więcej niż 10 razy i co najmniej 2.

Dostaniemy nazwy tych ID tych użytkowników, ich nazwę, reputację (https://www.wikihow.com/Gain-Reputation-on-Stack-Exchange), wiek i miejsce zamieszkania.

In [40]:
command6 = """
SELECT DISTINCT
    Users.Id,
    Users.DisplayName,
    Users.Reputation,
    Users.Age,
    Users.Location
FROM (
        SELECT
            Name, UserID
        FROM Badges
        WHERE Name IN (
                SELECT
                    Name
                FROM Badges
                WHERE Class=1
                GROUP BY Name
                HAVING COUNT(*) BETWEEN 2 AND 10
        )
        AND Class=1
    ) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id
"""
pd.read_sql_query(command6, conn)

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273,27.0,"London, UK"
1,19,VMAtm,18556,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360,NaN,Down underer
4,793,mindcorrosive,10531,32.0,Bulgaria
5,693,RoflcoptrException,33300,NaN,None
6,6669,Relaxed,69405,NaN,None
7,1737,Gayot Fow,70237,NaN,"London, United Kingdom"
8,39065,Pont,1004,NaN,Austria
9,19400,phoog,34342,50.0,"New York, NY"


In [41]:
Names = set(Badges.loc[Badges.Class == 1]\
.groupby(["Name"])\
.filter(lambda df: 2 <= len(df) <= 10)\
.Name.reset_index(drop=True)\
.values)

ValuableBadges = Badges.loc[[name in Names for name in Badges.Name] & (Badges.Class == 1), ["Name", "UserId"]]\
.reset_index(drop=True)

result = pd.merge(ValuableBadges, Users, left_on="UserId", right_on="Id")\
.loc[:, ["Id", "DisplayName", "Reputation", "Age", "Location"]]

result = result.drop_duplicates().reset_index(drop=True)
result

,Id,DisplayName,Reputation,Age,Location
0,108,Ankur Banerjee,31273,27.0,"London, UK"
1,19,VMAtm,18556,33.0,"Tampa, FL, United States"
2,101,Mark Mayo,121667,37.0,"Sydney, New South Wales, Australia"
3,466,iHaveacomputer,8360,NaN,Down underer
4,793,mindcorrosive,10531,32.0,Bulgaria
5,693,RoflcoptrException,33300,NaN,NaN
6,6669,Relaxed,69405,NaN,NaN
7,1737,Gayot Fow,70237,NaN,"London, United Kingdom"
8,39065,Pont,1004,NaN,Austria
9,19400,phoog,34342,50.0,"New York, NY"


**Uwaga:**
`Badges.loc[[name in Names for name in Badges.Name] & (Badges.Class == 1), ["Name", "UserId"]]` zadziała również bez nawiasów wokół `Badges.Class == 1` i dostaniemy wynik, ale nie taki jaki byśmy chcieli (dostaniemy więcej wierszy na wyjściu).

### Zadanie 7
Wybierzemy posty, które nie dostały żadnego lajka w roku 2016 i później, ale za to dużo lajków przed 2016 (na przykład przestarzałe i nieaktualne już pytania, które kiedyś były istotne (?)). Wybierzemy 10 tych pytań (tytuły) sortując po sumarycznej liczbie lajków i pokażemy tę liczbę lajków.

In [42]:
command7 = """
SELECT
    Posts.Title,
    VotesByAge2.OldVotes
FROM Posts
JOIN (
    SELECT
        PostId,
        MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
        MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
        SUM(Total) AS Votes
    FROM (
        SELECT
            PostId,
            CASE STRFTIME('%Y', CreationDate)
                WHEN '2017' THEN 'new'
                WHEN '2016' THEN 'new'
                ELSE 'old'
                END VoteDate,
            COUNT(*) AS Total
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, VoteDate
        ) AS VotesByAge
        GROUP BY VotesByAge.PostId
        HAVING NewVotes=0
) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
WHERE Posts.PostTypeId=1
ORDER BY VotesByAge2.OldVotes DESC
LIMIT 10
"""
pd.read_sql_query(command7, conn)

,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,What times of the year are best for visiting F...,23
7,What is the business model of commercial free ...,23
8,Getting work on a cruise ship in order to travel,23
9,Carrying medicines internationally for a friend,23


In [43]:
Votes["VoteDate"] = ['new' if year in {'2018', '2017', '2016'} else 'old' for year in Votes.CreationDate.str[0:4]]

VotesByAge = Votes.loc[Votes.VoteTypeId == 2, ['PostId', 'VoteDate']]\
.groupby(['PostId', 'VoteDate']).size().reset_index()
VotesByAge.columns = ['PostId', 'VoteDate', 'Total']

In [44]:
VotesByAge['NewVotes'] = VotesByAge.apply(lambda row: row.Total if row.VoteDate == 'new' else 0, axis=1)
VotesByAge['OldVotes'] = VotesByAge.apply(lambda row: row.Total if row.VoteDate == 'old' else 0, axis=1)

VotesByAge2 = VotesByAge.groupby(['PostId'])\
.agg({
    'NewVotes': max,
    'OldVotes': max,
    'Total': sum
})\
.reset_index()

VotesByAge2 = VotesByAge2.loc[VotesByAge2.NewVotes == 0].drop('NewVotes', axis=1)

In [45]:
result = pd.merge(Posts.loc[Posts.PostTypeId == 1], VotesByAge2, left_on="Id", right_on="PostId")

result = result.loc[:, ["Title", "OldVotes"]]\
.sort_values("OldVotes", ascending=False).head(10)\
.reset_index(drop=True)

result

,Title,OldVotes
0,Which European cities have bike rental station...,39
1,Why do hostels require you to 'rent' bedding?,28
2,What to do with your valuables on a low-cost h...,25
3,Can't check-in to a hotel because I am 18,25
4,What are some good ways to find things to expl...,24
5,Alarm Clock without Noise? To wake up in commo...,24
6,Should I avoid overnight flights from Dubai to...,23
7,"Can a visitor who is ""of age"" in their country...",23
8,What is the business model of commercial free ...,23
9,Carrying medicines internationally for a friend,23


**Uwagi:**
* Trochę mylący jest dla mnie argument axis. Mówi się, że `axis=1` dotyczy operacji na wierszach (tak jak na przykład w metodzie `apply()`, którą zastosowałem tworząc nową kolumnę na podstawie poprzednich), natomiast dajemy `axis=1` również w metodzie `drop()` gdy chcemy pozbyć się wybranej *kolumny*.

### Porównanie z Rem:
Ponieważ całkiem niedawno miałem do czynienia z R-owymi odpowiednikami pandasa, tj. dplyr i data.table mogę powiedziec kilka słów i moich opinii o różnicach między tymi pakietami.

Zaletą pakietów R-owych w porównaniu z pandasem jest moim zdaniem całkiem elegancki (zwłaszcza w dplyr) sposób pisania zapytań. Do tego łatwo w locie zmieniać nazwy kolumn na jakie się nam podoba i pisać zapytania w jednym logicznym ciągu. 

Pandas może nie jest tak elegancki, ale przynajmniej jest zrozumiały. Wywołujemy w nim metody na różnych obiektach (DataFrame, obiekt GroupBy). Natomiast to co się dzieje w dplyr mimo swojej czytelności jest dla mnie trochę magiczne i nie do końca to rozumiem. Zdziwiło mnie natomiast, że pandas mimo swojej popularności nie jest tak dopracowany jak myślałem. Brakuje mi możliwości grupowania po kolumnach zawierających NA, co w Rze nie było żadnym problemem.